In [0]:
import os
from dotenv import load_dotenv
from datetime import datetime

from pyspark.sql import functions as F

In [0]:
load_dotenv()

DEFAULT_PATH = os.getenv("DEFAULT_PATH")

In [0]:
_current_datetime = dbutils.widgets.get('current_datetime')
_rate = dbutils.widgets.get('rate')

# _current_datetime = '2025-09-10 00:40:54.522'
# _rate = 'BRL'

current_date = datetime.today().strftime('%Y-%m-%d')
# current_date = '2025-09-09'

raw_path = f"{DEFAULT_PATH}/Raw/{current_date}.json"
silver_path = f"{DEFAULT_PATH}/Silver/{current_date}.parquet"

In [0]:
df = spark.read.json(raw_path)

In [0]:
# A moeda que estiver no base_currency equivale a 1
df = df.withColumn('base_currency', F.lit(_rate))\
       .withColumn('timestamp', F.lit(_current_datetime).cast('timestamp'))\
       .withColumnRenamed('currency', 'moeda')\
       .withColumnRenamed('value', 'taxa')\
       .filter(F.col('moeda') != 'BRL')

if os.path.exists(silver_path):
       df.write.mode('append').format('parquet').save(silver_path)
else:
       df.write.coalesce(1).mode('overwrite').format('parquet').save(silver_path)
